<a href="https://colab.research.google.com/github/paradisepilot/study/blob/master/earth-observation/earth-engine/e2e-gee/e2e_gee_06_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

One of the most commonly asked questions by Earth Engine users is - *How do I download all images in a collection*? The Earth Engine Python API comes with a `ee.batch` module that allows you to launch batch exports and manage tasks. The recommended way to do batch exports like this is to use the Python API's `ee.batch.Export` functions and use a Python for-loop to iterate and export each image. The `ee.batch` module also gives you ability to control *Tasks* - allowing you to automate exports.

In [1]:
import ee

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=4HunUVzl4iTChcTvNe6KS3yJYoys9HwJZCcMSST0P_w&tc=wr_3kKr46eMx8zni9EfSuUnCegT6wGtYaM5ecLyzNho&cc=_pvqQn5rtg2iNAgtNILsVklduq1WxxkIhQaYknrtsZQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qhv8HD4VoWgeRl4WNv-yKKZg1g8DF3mypzFIoo_7jPF-n6508W15VA

Successfully saved authorization token.


In [3]:
ee.Initialize()

#### Create a Collection

In [4]:
geometry = ee.Geometry.Point([107.61303468448624, 12.130969369851766])
s2 = ee.ImageCollection("COPERNICUS/S2")
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.bounds(geometry)) \
  .map(maskS2clouds)

# Write a function that computes NDVI for an image and adds it as a band
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('ndvi')
  return image.addBands(ndvi)

withNdvi = filtered.map(addNDVI)

#### Export All Images

Exports are done via the ``ee.batch`` module. A key difference between javascript and Python version is that the `region` parameter needs to be supplied with actual geometry coordinates.

In [9]:
image_ids = withNdvi.aggregate_array('system:index').getInfo()
for i in range(0,len(image_ids)):
  print(i,": ", image_ids[i])

0 :  20190113T031059_20190113T031703_T48PYU
1 :  20190128T031001_20190128T031602_T48PYU
2 :  20190207T030911_20190207T031859_T48PYU
3 :  20190212T030839_20190212T032044_T48PYU
4 :  20190217T030801_20190217T032232_T48PYU
5 :  20190222T030729_20190222T031329_T48PYU
6 :  20190227T030651_20190227T031435_T48PYU
7 :  20190304T030619_20190304T032047_T48PYU
8 :  20190309T030541_20190309T032019_T48PYU
9 :  20190319T030541_20190319T032255_T48PYU
10 :  20190324T030539_20190324T031715_T48PYU
11 :  20190403T030549_20190403T032022_T48PYU
12 :  20190423T030549_20190423T031830_T48PYU
13 :  20190503T030549_20190503T032027_T48PYU
14 :  20190513T030549_20190513T032056_T48PYU
15 :  20190518T030551_20190518T031147_T48PYU
16 :  20190602T030549_20190602T032027_T48PYU
17 :  20190712T030549_20190712T032057_T48PYU
18 :  20190925T030541_20190925T032110_T48PYU
19 :  20190930T030539_20190930T031345_T48PYU
20 :  20191005T030601_20191005T030740_T48PYU
21 :  20191015T030701_20191015T030849_T48PYU
22 :  20191025T03081

In [6]:
# Export with 100m resolution for this demo
for i, image_id in enumerate(image_ids):
  image = ee.Image(withNdvi.filter(ee.Filter.eq('system:index', image_id)).first())
  task = ee.batch.Export.image.toDrive(**{
    'image': image.select('ndvi'),
    'description': 'Image Export {}'.format(i+1),
    'fileNamePrefix': image.id().getInfo(),
    'folder':'earthengine',
    'scale': 100,
    'region': image.geometry().bounds().getInfo()['coordinates'],
    'maxPixels': 1e10
  })
  task.start()
  print('Started Task: ', i+1)

Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7
Started Task:  8
Started Task:  9
Started Task:  10
Started Task:  11
Started Task:  12
Started Task:  13
Started Task:  14
Started Task:  15
Started Task:  16
Started Task:  17
Started Task:  18
Started Task:  19
Started Task:  20
Started Task:  21
Started Task:  22
Started Task:  23
Started Task:  24
Started Task:  25
Started Task:  26
Started Task:  27
Started Task:  28
Started Task:  29
Started Task:  30
Started Task:  31


#### Manage Running/Waiting Tasks

You can manage tasks as well. Get a list of tasks and get state information on them

In [7]:
tasks = ee.batch.Task.list()
for task in tasks:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

XFJNOR75ZRGHDVQEI3P2XFLR READY
BVSM65NSAUJGY42NAQNU7CJR READY
KW5O6EZACXGTVM4AFNDC54SN READY
WFMPWLMCIWQDV44AVCVY2MTG READY
Z7VDIKAIIVXCIJDC5WW7KMTV READY
ASUEZYJLW2KODG3X6YGRNTAD READY
3SVDL6K42EEEMVOMVPVGLLEX READY
3Y74Y4ZDMNW5LMEWFQ7V4ULT READY
B3JC5XB7GXT4UE3AWDIBTQ4Q READY
ABHJTC72Q3DQZVCSSKWCWJ7X READY
PG34AIDH3FP6A4VLCN5CXPQL READY
X2K55DYUUA2UGPCZCJVMXOPW READY
CM2RCRNVZMBX5WIYFYRVTHF5 READY
GX6MMR5XL4H2K3OFY3T3OA27 READY
GHNQCB5JTYEL3UDHHG7GEV5H READY
5WRL5RWKSWTBBDEWGCGKHSPE READY
Y4PXA4KIUBCC4RXKNRZP3BS4 READY
6SISE74A75DX723MF3KNUY5N READY
WWM3WZLBNTRC7QEEIOVRNAHM READY
RWEWZOOLZ2OBTJPI5ENIRQHB READY
TPVLWBQ3EWTYNYJ6IR5DQP55 RUNNING
TMTPBAXHKIOAGILA4SCFKHOV RUNNING
2IDKZJGKD4ZHSB6OH6ISDMJG RUNNING
5JZNETPCYG5S4GCVI4RZLILW RUNNING
KYWYGDMEHXJOBTSXRFYSRLV4 COMPLETED
3VDPGAVHXKJPRNUXUN3ZL3CH COMPLETED
5MZAR2VVALKTFMP46OOJWL4N COMPLETED
YD4VHQ4Y666JIWTD5MK4NOEA COMPLETED
J52B5URWLDT5LXYVNTZJNY6I COMPLETED
QRKMKAVOHFATQ2J5N6NHKZFO COMPLETED
OGIMJ2FKBSINIHPCOK4LV7BK COMPLETED
2W6

You can cancel tasks as well

In [ ]:
tasks = ee.batch.Task.list()
for task in tasks:
    task_id = task.status()['id']
    task_state = task.status()['state']
    if task_state == 'RUNNING' or task_state == 'READY':
        task.cancel()
        print('Task {} canceled'.format(task_id))
    else:
        print('Task {} state is {}'.format(task_id, task_state))

### Exercise

The code below uses the TerraClimate data and creates an ImageCollection with 12 monthly images of maximum temperature. It also extract the geometry for Australia from the LSIB collection. Add the code to start an export task for each image in the collection for australia.

- **Hint1**: TerraClimate images have a scale of 4638.3m
- **Hint2**: You need to export the image contained in the clippedImage variable

In [10]:
import ee

lsib = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
australia = lsib.filter(ee.Filter.eq('country_na', 'Australia'))
geometry = australia.geometry()

terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
tmax = terraclimate.select('tmmx')

def scale(image):
  return image.multiply(0.1) \
    .copyProperties(image,['system:time_start'])

tmaxScaled = tmax.map(scale)

filtered = tmaxScaled \
  .filter(ee.Filter.date('2020-01-01', '2021-01-01')) \
  .filter(ee.Filter.bounds(geometry))

image_ids = filtered.aggregate_array('system:index').getInfo()
print('Total images: ', len(image_ids))
for i, image_id in enumerate(image_ids):
  print(i,": ",image_id)

Total images:  12
0 :  202001
1 :  202002
2 :  202003
3 :  202004
4 :  202005
5 :  202006
6 :  202007
7 :  202008
8 :  202009
9 :  202010
10 :  202011
11 :  202012


Replace the comments with your code.

In [26]:
for i, image_id in enumerate(image_ids):

    exportImage = ee.Image(filtered.filter(ee.Filter.eq('system:index', image_id)).first())
  
    # Clip the image to the region geometry
    clippedImage = exportImage.clip(geometry)
    
    ## Create the export task using ee.batch.Export.image.toDrive()
    task = ee.batch.Export.image.toDrive(**{
       'image': clippedImage.select('tmmx'),
       'description': 'Image Export {}'.format(i+1),
        'fileNamePrefix': clippedImage.id().getInfo(),
        'folder':'earthengine',
        'scale': 1000,
        'region': clippedImage.geometry().bounds().getInfo()['coordinates'],
        'maxPixels': 1e10
    })

    ## Start the task
    task.start()
    print('Started Task: ', i+1)


Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7
Started Task:  8
Started Task:  9
Started Task:  10
Started Task:  11
Started Task:  12


In [30]:
tasks = ee.batch.Task.list()
for task in tasks:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

NTLWG2EEAZQDG6G3NKP23TNX READY
QW5RCVW3JHKSEY5ZY3EXFRSD READY
CGLGOENNDPU2UHRFKC7NR2VH READY
ODTIXUQI4I5JCE7XFJHZAACZ READY
CDEVE6LDAEBYMPUVUHWWYB75 READY
AHV32DW5VJBIDX2II6MQQB6X READY
3LKT6KHMMYDVKMTB3ZNQB3MP READY
NDQ25P3I3VUYELHPROHIDMGK READY
7QTZNAZ46GHI62Q23AAOTRKJ READY
I4VTMI4CJGZISKPD3SDR67QQ READY
553LLZKDDGU4UNGZCNTV5KAG READY
ZWQIM5EYLAOTMT6VZQGQNY6B READY
SN46GMOUDXOESZGMHUUSZD2X CANCELLED
4Y32LTQ24NB7BYVXO5ZDGFPH CANCELLED
Q6IFW3FHGNTE2QAJHN7TZ3FE CANCELLED
7I36GYXQSFFOVSMKOK33HSRW CANCELLED
S2Z5247QK6QEG5BMO5JYS7BR CANCELLED
XEBYIFSHRN6DV5SVK6DR4YP5 CANCELLED
H5ZLL6563G22QBDLSDPGRK2F CANCELLED
3KO2CQCIYXUHJFXW56R5HAE3 CANCELLED
JPZLOCNATBQQO2GQXCD5IFW6 CANCEL_REQUESTED
DGNVNVAPBMPZ4UPZEMZKG2AF CANCEL_REQUESTED
LAVLGPWWMFGZAFZZOME7GZFC CANCEL_REQUESTED
7FJRPYMQDGMR7J3TLGIAGQLK CANCEL_REQUESTED
XFJNOR75ZRGHDVQEI3P2XFLR COMPLETED
BVSM65NSAUJGY42NAQNU7CJR COMPLETED
KW5O6EZACXGTVM4AFNDC54SN COMPLETED
WFMPWLMCIWQDV44AVCVY2MTG COMPLETED
Z7VDIKAIIVXCIJDC5WW7KMTV COMPLETED
ASUEZ

In [22]:
tasks = ee.batch.Task.list()
for task in tasks:
    task_id = task.status()['id']
    task_state = task.status()['state']
    if task_state == 'RUNNING' or task_state == 'READY':
        task.cancel()
        print('Task {} canceled'.format(task_id))
    else:
        print('Task {} state is {}'.format(task_id, task_state))

Task SN46GMOUDXOESZGMHUUSZD2X canceled
Task 4Y32LTQ24NB7BYVXO5ZDGFPH canceled
Task Q6IFW3FHGNTE2QAJHN7TZ3FE canceled
Task 7I36GYXQSFFOVSMKOK33HSRW canceled
Task S2Z5247QK6QEG5BMO5JYS7BR canceled
Task XEBYIFSHRN6DV5SVK6DR4YP5 canceled
Task H5ZLL6563G22QBDLSDPGRK2F canceled
Task 3KO2CQCIYXUHJFXW56R5HAE3 canceled
Task JPZLOCNATBQQO2GQXCD5IFW6 canceled
Task DGNVNVAPBMPZ4UPZEMZKG2AF canceled
Task LAVLGPWWMFGZAFZZOME7GZFC canceled
Task 7FJRPYMQDGMR7J3TLGIAGQLK canceled
Task XFJNOR75ZRGHDVQEI3P2XFLR state is COMPLETED
Task BVSM65NSAUJGY42NAQNU7CJR state is COMPLETED
Task KW5O6EZACXGTVM4AFNDC54SN state is COMPLETED
Task WFMPWLMCIWQDV44AVCVY2MTG state is COMPLETED
Task Z7VDIKAIIVXCIJDC5WW7KMTV state is COMPLETED
Task ASUEZYJLW2KODG3X6YGRNTAD state is COMPLETED
Task 3SVDL6K42EEEMVOMVPVGLLEX state is COMPLETED
Task 3Y74Y4ZDMNW5LMEWFQ7V4ULT state is COMPLETED
Task B3JC5XB7GXT4UE3AWDIBTQ4Q state is COMPLETED
Task ABHJTC72Q3DQZVCSSKWCWJ7X state is COMPLETED
Task PG34AIDH3FP6A4VLCN5CXPQL state is COM